In [ ]:
# <colab에서 실행>

# 기본 라이브러리 임포트
import pandas as pd
import numpy as np
import glob # glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다.

In [ ]:
# 구글 드라이브와 연동한다.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# audio 폴더를 만든다.
import os
os.mkdir('/content/drive/MyDrive/ColabNotebooks/audio')

In [ ]:
# ColabNotebooks에서 audio 데이터를 zip으로 업로드 후 audio 폴더에 압축을 푼다.
!unzip -qq /content/drive/MyDrive/ColabNotebooks/audio.zip  -d /content/drive/MyDrive/ColabNotebooks/audio

In [ ]:
# 모든 개와 고양이 소리가 있는 위치+파일명을 X_path에 넣는 작업을 한다.
import tensorflow as tf

# seed값 설정
tf.random.set_seed(777)

Test_root_b = glob.glob('/content/drive/MyDrive/ColabNotebooks/audio/birds')[0] 
Test_root_c = glob.glob('/content/drive/MyDrive/ColabNotebooks/audio/cats')[0] 
Test_root_d = glob.glob('/content/drive/MyDrive/ColabNotebooks/audio/dogs')[0] 

print(Test_root_b) #bird 경로 담기
print(Test_root_c) #cats 경로 담기
print(Test_root_d) #dogs 경로 담기

In [ ]:
# 모든 개, 고양이, 새 소리가 있는 위치와 파일명을 X_path 에 넣는 작업을 한다.
X_path = glob.glob(Test_root_b+"/test/*")
X_path = X_path + glob.glob(Test_root_c + "/test/*")
X_path = X_path + glob.glob(Test_root_d + "/test/*")
X_path = X_path + glob.glob(Test_root_b + "/train/*")
X_path = X_path + glob.glob(Test_root_c + "/train/*")
X_path= X_path + glob.glob(Test_root_d + "/train/*")

In [ ]:
# 정답 데이터 만드는 코드(새는 0, 고양이는 1, 강아지는 2로 지정)

import ntpath # 특정 경로에서 파일들을 가져오는 라이브러리
y = np.empty((0, 1, )) 
for f in X_path:
    if 'birds' in f: #  음성 데이터가 있는 디렉토리의 데이터가 새 음성 데이터라면 
        resp = np.array([0])  #   [0]  , 1차원
        resp = resp.reshape(1, 1, ) # [[0]]  , 2차원 --> 추후 원핫인코딩에 넣기 위해서
        y = np.vstack((y, resp))  # array([[0.]]) , 배열을 세로로 결합 , 데이터가 세로로 쌓인다. 
    elif 'cats' in f: # 음성 데이터가 있는 디렉토리의 데이터가 고양이 음성 데이터라면
        resp = np.array([1])  # [1] 
        resp = resp.reshape(1, 1, ) #[[1]]
        y = np.vstack((y, resp)) # 
    elif 'dogs' in f: # 음성 데이터가 있는 디렉토리의 데이터가 개 음성 데이터라면
        resp = np.array([2])  # [2] 
        resp = resp.reshape(1, 1, ) #[[2]]
        y = np.vstack((y, resp)) # 

print(len(y)) #1084

In [ ]:
# 훈련 데이터와 테스트 데이터를 분리한다.
# 훈련 90%, 테스트 10% 로 구성한다.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_path, y, test_size=0.1, random_state=42)

print(len(X_train)) # 975
print(len(X_test))  # 109
print(len(y_train)) # 975
print(len(y_test)) # 109

In [ ]:
# 개, 고양이, 새 음성에서 진폭을 추출해서 리스트에 담는 함수를 생성한다.
    
import librosa # 음원 데이터에서 신경망 학습을 시킬 수 있는 데이터를 뽑아내는 모듈

def librosa_read_wav_files(wav_files):
    if not isinstance(wav_files, list): 
        wav_files = [wav_files]  # 리스트화 
    return [librosa.load(f)[0] for f in wav_files] # 진폭과 샘플레이트중 진폭만 담는다. 

wav_rate = librosa.load(X_train[0])[1] 
wav_rate  # 훈련 데이터의 첫번째 동물의 sample rate 값을 wave_rate 에 담는다.

X_train = librosa_read_wav_files(X_train)  # 훈련 데이터 975개의 진폭 데이터를 X_train 담는다.
X_test  = librosa_read_wav_files(X_test)   # 테스트 데이터 109개의 진폭 데이터를 X_test 에 담는다.

print(len(X_train))  # 975개의 진폭 데이터 리스트
print(len(X_test))   # 109개의 진폭 데이터 리스트 

In [ ]:
# 음성 특징 추출 함수
def extract_features(audio_samples, sample_rate): 
    extracted_features = np.empty((0, 61, )) 
    if not isinstance(audio_samples, list):
        audio_samples = [audio_samples] # 리스트화 한다.
    for sample in audio_samples:  # 진폭 데이터 리스트를 하나씩 가져온다.
        zero_cross_feat = librosa.feature.zero_crossing_rate(sample).mean() 
        mfccs = librosa.feature.mfcc(y=sample, sr=sample_rate, n_mfcc=60) 
        mfccsscaled = np.mean(mfccs.T,axis=0)  
        mfccsscaled = np.append(mfccsscaled, zero_cross_feat)  
        mfccsscaled = mfccsscaled.reshape(1, 61, )  
        extracted_features = np.vstack((extracted_features, mfccsscaled)) 
    return extracted_features

In [ ]:
# train과 test 데이터를 음성 특징 추출하여 담는다.
X_train_features = extract_features(X_train, wav_rate) 
X_test_features  = extract_features(X_test, wav_rate)
print(len(X_train_features))
print(len(X_test_features))